# Explore trip clustering using DBSCAN

In `Radius Selection Unrolled`, we explored the options to select the radius based on distances around the start or end location. Can we also combine them to create a trip-level clustering that is an alternate, and much simpler implementation of the similarity code? Let's see if we can use DBSCAN to do this and whether the final trip counts are principled

### First, we read the data and extract the most common purpose labels

In [ ]:
import pandas as pd
import numpy as np
import geojson as gj
import sklearn.cluster as sc
import sklearn.metrics.pairwise as smp

In [ ]:
import json
import copy

In [ ]:
import folium
import branca.element as bre

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as pltc

In [ ]:
from IPython import display
from uuid import UUID

import bson.json_util as bju
import bson.objectid as boi

In [ ]:
import emission.storage.timeseries.abstract_timeseries as esta
import emission.storage.decorations.trip_queries as esdtq

### Read data and setup variables

In [ ]:
all_users = esta.TimeSeries.get_uuid_list()
confirmed_trip_df_map = {}
labeled_trip_df_map = {}
expanded_trip_df_map = {}
for u in all_users:
    ts = esta.TimeSeries.get_time_series(u)
    ct_df = ts.get_data_df("analysis/confirmed_trip")
    confirmed_trip_df_map[u] = ct_df
    labeled_trip_df_map[u] = esdtq.filter_labeled_trips(ct_df)
    expanded_trip_df_map[u] = esdtq.expand_userinputs(labeled_trip_df_map[u])

In [ ]:
n_trips_df = pd.DataFrame([[u, len(confirmed_trip_df_map[u]), len(labeled_trip_df_map[u])] for u in all_users], columns=["user_id", "all_trips", "labeled_trips"]); n_trips_df

In [ ]:
median_user = n_trips_df[n_trips_df.labeled_trips == n_trips_df.labeled_trips.median()].user_id.iloc[0]; median_user

In [ ]:
median_user_df = expanded_trip_df_map[median_user]

In [ ]:
FINAL_RADIUS = 500
FINAL_POINT_DBSCAN = sc.DBSCAN(FINAL_RADIUS, min_samples=2, metric="precomputed")
FINAL_TRIP_DBSCAN = sc.DBSCAN(FINAL_RADIUS * 2, min_samples=2, metric="precomputed")

### Standard functions (currently copied over from other notebooks; should be refactored into a python file)

In [ ]:
def get_loc_df(loc_series):
    loc_df = pd.DataFrame(loc_series.apply(lambda p: p["coordinates"]).to_list(), columns=["longitude", "latitude"])
    # display.display(end_loc_df.head())
    return loc_df

In [ ]:
def get_distance_matrix(loc_df):
    EARTH_RADIUS = 6371000
    radians_lat_lon = np.radians(loc_df[["latitude", "longitude"]])
    dist_matrix_meters = pd.DataFrame(smp.haversine_distances(radians_lat_lon, radians_lat_lon) * 6371000)
    return dist_matrix_meters

### Approach 1: Recluster based on trip distance

- We add the start and end distances to form a combined distance matrix.
- We cluster with a radius that is twice what we had before

One potential challenge is that we may have one end of the trip be a very close match and the other end be very far (e.g. 10, 900) and still have the trip as a whole fit within the distance threshold (1000).

On the other hand, maybe that is OK - if both the start and the end match, then it must be a pretty good match, and maybe we can be a bit lax.

In [ ]:
start_distance_matrix = get_distance_matrix(get_loc_df(median_user_df.start_loc))
end_distance_matrix = get_distance_matrix(get_loc_df(median_user_df.end_loc))
start_loc_model = copy.copy(FINAL_POINT_DBSCAN).fit(start_distance_matrix)
end_loc_model = copy.copy(FINAL_POINT_DBSCAN).fit(end_distance_matrix)

In [ ]:
start_loc_model.labels_

In [ ]:
end_loc_model.labels_

In [ ]:
median_user_df["start_loc_cluster"] = start_loc_model.labels_
median_user_df["end_loc_cluster"] = end_loc_model.labels_

#### Try to calculate trip clusters by adding up the distances and clustering

In [ ]:
combined_distance_matrix = start_distance_matrix + end_distance_matrix
trip_model = copy.copy(FINAL_TRIP_DBSCAN).fit(combined_distance_matrix)

In [ ]:
trip_model.labels_

In [ ]:
median_user_df["trip_cluster_method1"] = trip_model.labels_

In [ ]:
np.count_nonzero(median_user_df.trip_cluster_method1 != -1)

In [ ]:
np.count_nonzero(trip_model.labels_ != -1), len(trip_model.labels_)

### Approach 2: Find trips whose start and end location are both in clusters

- Find all combinations of start and end clusters
- Retain only ones where both start and end are non-noisy
- Group them to get a unique set of (start, end) tuples and treat them (represented by an index) as the cluster labels
- For each (start,end) tuple assign the corresponding cluster label

In [ ]:
quick_upper_bound = median_user_df.query("start_loc_cluster != -1 and end_loc_cluster != -1"); len(quick_upper_bound)

We just got a quick upper bound on the number of trips by this method. Note that we just know that both the start and end start in a cluster, we are not yet sure they start in **the same cluster**

In [ ]:
all_combos = median_user_df.groupby(["start_loc_cluster", "end_loc_cluster"])
valid_combos = [p for p in all_combos.groups if p[0] != -1 and p[1] != -1]

In [ ]:
(len(list(all_combos.groups)), len(valid_combos))

In [ ]:
# dict(all_combos.groups)

In [ ]:
dict(all_combos.groups)[(-1,-1)], dict(all_combos.groups)[(2,2)]

In [ ]:
all_combos_dict = dict(all_combos.groups)
valid_combos_series = pd.Series(valid_combos)

for g, idxlist in all_combos_dict.items():
    print(g, idxlist)
    match = valid_combos_series[valid_combos_series == g]
    if len(match) == 0:
        print(f"invalid combo {g} found for entries {idxlist}, trip is not in a cluster")
        median_user_df.loc[idxlist, "trip_cluster_method2"] = -1
    else:
        print(f"valid combo {g} found for entries {idxlist}, setting trip cluster to {match.index[0]}")
        median_user_df.loc[idxlist, "trip_cluster_method2"] = int(match.index[0])

In [ ]:
median_user_df[["trip_cluster_method1", "trip_cluster_method2"]]

In [ ]:
mismatch_df = median_user_df.query("(trip_cluster_method1 == -1 and trip_cluster_method2 != -1) or (trip_cluster_method1 != -1 and trip_cluster_method2 == -1)")

In [ ]:
np.count_nonzero(median_user_df.trip_cluster_method1 == -1), np.count_nonzero(median_user_df.trip_cluster_method2 == -1) 

There are actually *fewer* "noise" entries with method 2, so more clusters. let's do some additional validation.

First, we just compare the number of clusters.

In [ ]:
len(median_user_df.trip_cluster_method1.unique()), len(median_user_df.trip_cluster_method2.unique())

Whoa! Method 1 seems to generate bigger clusters, which, of course, may be good or bad. Next, we check that for each trip cluster, the start and end clusters are the same. So we can verify that each cluster of trips is between the same start and end points.

In [ ]:
np.count_nonzero(median_user_df.groupby("trip_cluster_method1").apply(lambda df: len(df.start_loc_cluster.unique()) == 1 and len(df.end_loc_cluster.unique()) == 1)), len(median_user_df.trip_cluster_method1.unique())

In [ ]:
np.count_nonzero(median_user_df.groupby("trip_cluster_method2").apply(lambda df: len(df.start_loc_cluster.unique()) == 1 and len(df.end_loc_cluster.unique()) == 1)), len(median_user_df.trip_cluster_method2.unique())

Aha! This is a clear argument for using the second method. In both methods, the noisy trips are not classified, as we might expect. However, in method 1, multiple trip clusters don't actually have the same start and end points. In method 2, everything other than the noisy trips starts and ends in the same point (by definition). But the flip side is that we have a lot more (small) clusters, so we will need to ask the user more often.

### Given that we are going to use method 2, maybe we are done?

Let's visualize a few trips to verify, both on a map and using matplotlib

In [ ]:
def get_geojson_for_trip_cluster(exp_df):
    print(len(cluster_trips))
    # [[[X1, Y1], [X1, Y1]],
    # [[X1, Y1], [X1, Y1]]]
    clistarray = cluster_trips[["start_loc", "end_loc"]].apply(
                    lambda se: [p["coordinates"] for p in se]).to_numpy().tolist()
    print([len(clist) for clist in clistarray])
    linestrings = [gj.LineString(coordinates=clist) for clist in clistarray]
    purpose_locs = gj.FeatureCollection(cluster_trips.start_loc.to_list() +
                                        cluster_trips.end_loc.to_list() +
                                        linestrings)
    return folium.features.GeoJson(purpose_locs)

In [ ]:
def get_geojson_for_point_cluster(exp_df, loc_field, loc_cluster_field, cluster_label):
    cluster_trips = exp_df[exp_df[loc_cluster_field] == cluster_label]
    print(len(cluster_trips))
    purpose_locs = gj.FeatureCollection(cluster_trips[loc_field].to_list())
    return folium.features.GeoJson(purpose_locs)

This user does have "home" as the most common purpose, but "transit transfer" and "personal med" are above "work". Let's start with focusing on home.

In [ ]:
fig = bre.Figure()
fig.add_subplot(1,2,1).add_child(folium.Map().add_child(get_geojson_for_trip_cluster(median_user_df[median_user_df["trip_cluster_method1"] == 0])))
fig.add_subplot(1,2,2).add_child(folium.Map().add_child(get_geojson_for_trip_cluster(median_user_df[median_user_df["trip_cluster_method2"] == 0])))

uh-oh, method1 was expected to be a bit bad, but method2 appears to be just as bad. While most of the trips are between two clear clusters, there are also some clear outliers, like the big vertical line. Let's visualize the start location clusters, both on a map and on a plot.

#### Cluster 0 on a map

In [ ]:
fig = bre.Figure()
fig.add_subplot(1,2,1).add_child(folium.Map().add_child(get_geojson_for_point_cluster(median_user_df, "start_loc", "start_loc_cluster", 0)))
fig.add_subplot(1,2,2).add_child(folium.Map().add_child(get_geojson_for_point_cluster(median_user_df, "end_loc", "end_loc_cluster", 0)))

In [ ]:
c0_method2_df = median_user_df[median_user_df.trip_cluster_method2 == 0]
c0_start_distance_matrix = start_distance_matrix.loc[c0_method2_df.index, c0_method2_df.index]
c0_start_distance_matrix[c0_start_distance_matrix > FINAL_RADIUS]

The distance matrix helps us see what happened. Filtering out only the distances > the radius, we see that there are some clear outliers, which are near each other, far from the other cluster, but one of the points is close enough to one of the outliers, and they get merged

In [ ]:
def get_loc_df_for_cluster(exp_df, loc_cluster_field, cluster_label, loc_field="end_loc"):
    # Reuse the same function to get the loc_df
    cluster_trips = exp_df[exp_df[loc_cluster_field] == cluster_label]
    return pd.concat([cluster_trips, pd.DataFrame(cluster_trips[loc_field].apply(lambda p: p["coordinates"]).to_list(), columns=["longitude", "latitude"])], axis=1)

In [ ]:
fig = plt.Figure(figsize=(20,5))
axarr = fig.subplots(1,3,sharex=True,sharey=True)
# end_loc_df.plot(kind="scatter", x="longitude", y="latitude", color = end_loc_df["90%"].apply(lambda c: colors[c]), ax=ax, colorbar=False)
get_loc_df_for_cluster(median_user_df, "start_loc_cluster", 0, "start_loc").plot(kind="scatter", x="longitude", y="latitude", ax=axarr[0])
get_loc_df_for_cluster(median_user_df, "start_loc_cluster", 1, "start_loc").plot(kind="scatter", x="longitude", y="latitude", ax=axarr[1])
get_loc_df_for_cluster(median_user_df, "start_loc_cluster", 2, "start_loc").plot(kind="scatter", x="longitude", y="latitude", ax=axarr[2])
# end_loc_df.plot(kind="scatter", x="longitude", y="latitude", c = "95%", cmap=get_colormap(end_loc_df["95%"]), ax=ax, colorbar=False)
# ax = fig.add_subplot(1,3,3)
# end_loc_df.plot(kind="scatter", x="longitude", y="latitude", c = "99%", cmap=get_colormap(end_loc_df["99%"]), ax=ax, colorbar=False)
fig

Judging from this, clusters 2 and 3 might give better results. Let's try to verify that before trying to figure out how to fix this

In [ ]:
fig = bre.Figure()
fig.add_subplot(1,2,1).add_child(folium.Map().add_child(get_geojson_for_trip_cluster(median_user_df[median_user_df["trip_cluster_method1"] == 1])))
fig.add_subplot(1,2,2).add_child(folium.Map().add_child(get_geojson_for_trip_cluster(median_user_df[median_user_df["trip_cluster_method2"] == 1])))

In [ ]:
fig = bre.Figure()
fig.add_subplot(1,2,1).add_child(folium.Map().add_child(get_geojson_for_trip_cluster(median_user_df[median_user_df["trip_cluster_method1"] == 2])))
fig.add_subplot(1,2,2).add_child(folium.Map().add_child(get_geojson_for_trip_cluster(median_user_df[median_user_df["trip_cluster_method2"] == 2])))

So this is another good indication that method 2 is better than method 1. It looks like this problem has been asked but not answered on stackoverflow.

https://stackoverflow.com/questions/48217127/distance-based-classification

I can think of an approach to use repeated iterations of DBSCAN.

In [ ]:
c1_df = median_user_df[median_user_df.trip_cluster_method2 == 1]
c1_start_distance_matrix = start_distance_matrix.loc[c1_df.index, c1_df.index]
c1_start_distance_matrix[c1_start_distance_matrix > FINAL_RADIUS]

In [ ]:
[p for p in c0_method2_df.index if p not in start_loc_model.core_sample_indices_]

In [ ]:
len(start_loc_model.core_sample_indices_), len(np.unique(start_loc_model.labels_)), len(median_user_df.start_loc_cluster.unique())